In [3]:
import unstructured.partition
import os

In [2]:
from IPython.display import JSON

import json

from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError

from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import dict_to_elements, elements_to_json

In [13]:
from unstructured.partition.pdf import partition_pdf

# Specify the path to your PDF file
filename = os.path.join('..', 'data', '')

# Call the partition_pdf function
# Returns a List[Element] present in the pages of the parsed pdf document
elements = partition_pdf(filename)

# Now, elements is a list of all elements present in the pages of the parsed pdf document

In [14]:

len(elements)

2053

In [15]:
element_dict = [el.to_dict() for el in elements]
output = json.dumps(element_dict, indent=2)
# print(output)

In [16]:
unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

{'UncategorizedText', 'NarrativeText', 'ListItem', 'Title'}


In [19]:
# 1st way

# from unstructured.partition.auto import partition

# elements = partition(filename=filename,
#                      strategy='hi_res',
#            )

# tables = [el for el in elements if el.category == "Table"]

# print(tables[0].text)
# print(tables[0].metadata.text_as_html)

In [20]:
# 2nd way 

# from unstructured.partition.pdf import partition_pdf

# elements = partition_pdf(filename=filename,
#                          infer_table_structure=True,
#                          strategy='hi_res',
#            )

# tables = [el for el in elements if el.category == "Table"]

# print(tables[0].text)
# print(tables[0].metadata.text_as_html)


#### ERROR - TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

 - For SAAS API, visit this - https://unstructured.io/api-key-hosted
 - For Free Unstructured API, visit this - https://docs.unstructured.io/api-reference/api-services/free-api

In [ ]:
%pip install python-dotenv

In [ ]:


import os
from dotenv import load_dotenv, find_dotenv

# Load the .env file
load_dotenv(find_dotenv())

free_api_key_auth = os.environ.get('free_api_key_auth')

client = UnstructuredClient(
    api_key_auth=free_api_key_auth
)


# 3rd way 

with open(filename, "rb") as f:
    files=shared.Files(
        content=f.read(),
        file_name=filename,
    )

req = shared.PartitionParameters(
    files=files,
    strategy="hi_res",
    hi_res_model_name="yolox",
    skip_infer_table_types=[],
    pdf_infer_table_structure=True,
)

try:
    resp = client.general.partition(req)
    elements = dict_to_elements(resp.elements)
except SDKError as e:
    print(e)

In [ ]:
tables = [el for el in elements if el.category == "Table"]

In [ ]:
len(tables)

In [ ]:
tables[0].text

In [ ]:
# view what the HTML in the metadata field looks like

from io import StringIO 
from lxml import etree

table_html = tables[0].metadata.text_as_html

parser = etree.XMLParser(remove_blank_text=True)
file_obj = StringIO(table_html)
tree = etree.parse(file_obj, parser)
print(etree.tostring(tree, pretty_print=True).decode())

# let's display this table
from IPython.core.display import HTML
HTML(table_html)

In [ ]:
%pip install langchain langchain_core langchain_community

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.documents import Document
from langchain.chains.summarize import load_summarize_chain

In [ ]:
llm = ChatOllama(model="llama3")
chain = load_summarize_chain(llm, chain_type="stuff")
output = chain.invoke([Document(page_content=table_html)])

In [ ]:
import pandas as pd

# Convert HTML table to pandas DataFrame
dfs = pd.read_html(table_html)

In [ ]:
# Assuming there's only one table, get the DataFrame
df = dfs[0]

# Now you have the DataFrame
print(df)

In [ ]:
df.head()